In [78]:
import numpy as np
np.random.seed(0)

P = [
    [0.9, 0.1, 0.0, 0.0, 0.0, 0.0],
    [0.5, 0.0, 0.5, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.6, 0.0, 0.4],
    [0.0, 0.0, 0.0, 0.0, 0.3, 0.7],
    [0.0, 0.2, 0.3, 0.5, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
]

P = np.array(P)

rewards = [-1, -2, -2, 10, 1, 0] # rewards for states 0 to 5
gamma = 0.5 # discount factor

In [79]:
def compute_return(start_index, chain, gamma):
    G = 0
    for i in reversed(range(start_index, len(chain))): # from the end to the stratt_index of chain
        G = gamma * G + rewards[chain[i] - 1] # -1 for zero-based index

    return G

In [80]:
chain = [1, 2, 3, 6]
start_index = 0
G = compute_return(start_index, chain, gamma)
print('Return G starting from index %d of chain: %f' % (start_index, G))

Return G starting from index 0 of chain: -2.500000


In [81]:
def compute(P, rewards, gamma, states_num):
    rewards = np.array(rewards).reshape((-1, 1))
    # np.linalg.inv: matrix inverse
    # np.eye: identity matrix
    value = np.dot(np.linalg.inv(np.eye(states_num, states_num) - gamma * P), rewards)

    return value

In [82]:
V = compute(P, rewards, gamma, states_num=6)
print('Value function V: \n', V)

Value function V: 
 [[-2.01950168]
 [-2.21451846]
 [ 1.16142785]
 [10.53809283]
 [ 3.58728554]
 [ 0.        ]]


In [83]:
S = ['s1', 's2', 's3', 's4', 's5'] # set of states
A = ['Keep s1', 'Go s1', 'Go s2', 'Go s3', 'Go s4', 'Go s5', 'Go in Prob'] # set of actions
P = { # state transition func
    "s1-Keep s1-s1": 1.0,
    "s1-Go s2-s2": 1.0,
    "s2-Go s1-s1": 1.0,
    "s2-Go s3-s3": 1.0,
    "s3-Go s4-s4": 1.0,
    "s3-Go s5-s5": 1.0,
    "s4-Go s5-s5": 1.0,
    "s4-Go in Prob-s2": 0.2,
    "s4-Go in Prob-s3": 0.4,
    "s4-Go in Prob-s4": 0.4,
}
R = { # reward func
    "s1-Keep s1": -1,
    "s1-Go s2": 0,
    "s2-Go s1": -1,
    "s2-Go s3": -2,
    "s3-Go s4": -2,
    "s3-Go s5": 0,
    "s4-Go s5": 10,
    "s4-Go in Prob": 1,
}
gamma = 0.5 # discount factor
MDP = (S, A, P, R, gamma) # markov decision process

In [84]:
# Policy 1, Random Policy
Pi_1 = {
    "s1-Keep s1": 0.5,
    "s1-Go s2": 0.5,
    "s2-Go s1": 0.5,
    "s2-Go s3": 0.5,
    "s3-Go s4": 0.5,
    "s3-Go s5": 0.5,
    "s4-Go s5": 0.5,
    "s4-Go in Prob": 0.5,
}
# Policy 2
Pi_2 = {
    "s1-Keep s1": 0.6,
    "s1-Go s2": 0.4,
    "s2-Go s1": 0.3,
    "s2-Go s3": 0.7,
    "s3-Go s4": 0.5,
    "s3-Go s5": 0.5,
    "s4-Go s5": 0.1,
    "s4-Go in Prob": 0.9,
}

def join(str1, str2):
    return str1 + '-' + str2

In [85]:
gamma = 0.5
# MDP to MRP
P_from_mdp_to_mrp = [
    [0.5, 0.5, 0.0, 0.0, 0.0],
    [0.5, 0.0, 0.5, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.5, 0.5],
    [0.0, 0.1, 0.2, 0.2, 0.5],
    [0.0, 0.0, 0.0, 0.0, 1.0],
]
P_from_mdp_to_mrp = np.array(P_from_mdp_to_mrp)
R_from_mdp_to_mrp = [-0.5, -1.5, -1.0, 5.5, 0]

V = compute(P_from_mdp_to_mrp, R_from_mdp_to_mrp, gamma, 5)
print("State Value of MDP", V)

State Value of MDP [[-1.22555411]
 [-1.67666232]
 [ 0.51890482]
 [ 6.0756193 ]
 [ 0.        ]]


In [277]:
### Monte Carlo Simulation for Policy Evaluation
def sample(MDP, Pi, timestep_max, number):
    S, A, P, R, gamma = MDP
    episodes = []
    for _ in range(number):
        episode = []
        timestep = 0
        state = S[np.random.randint(4)] # s5 is terminal state
        while timestep <= timestep_max and state != 's5':
            timestep += 1
            temp = 0
            rand = np.random.rand()
            for a in A: # choose action according to policy Pi
                temp += Pi.get(join(state, a), 0) # get the prob of action a under state
                if temp >= rand: # random choose action according to prob
                    action = a
                    reward = R.get(join(state, a), 0)
                    break
            temp = 0
            rand = np.random.rand()
            for s in S: # choose next state according to state transition func P
                temp += P.get(join(join(state, action), s), 0)
                if temp >= rand: # random choose next state according to prob
                    next_state = s
                    break
            episode.append((state, action, reward, next_state))
            state = next_state
        episodes.append(episode)

    return episodes


In [287]:
episodes = sample(MDP, Pi_1, 20, 5)
print('Firts Sequence of Episodes: \n', episodes[0])

Firts Sequence of Episodes: 
 [('s1', 'Keep s1', -1, 's1'), ('s1', 'Keep s1', -1, 's1'), ('s1', 'Go s2', 0, 's2'), ('s2', 'Go s1', -1, 's1'), ('s1', 'Keep s1', -1, 's1'), ('s1', 'Go s2', 0, 's2'), ('s2', 'Go s3', -2, 's3'), ('s3', 'Go s4', -2, 's4'), ('s4', 'Go s5', 10, 's5')]


In [295]:
def eval_policy(episodes, V, N, gamma):
    for episode in episodes:
        G = 0
        for t in reversed(range(len(episode))):
            state, action, reward, next_state = episode[t]
            G = gamma * G + reward
            N[state] += 1
            V[state] += (G - V[state]) / N[state] # incremental mean

    return None


timestep_max = 20
episodes = sample(MDP, Pi_1, timestep_max, 1000)
gamma = 0.5
V = {s: 0 for s in S}
N = {s: 0 for s in S}
eval_policy(episodes, V, N, gamma)
V

{'s1': -1.2253273289232065,
 's2': -1.665252654265004,
 's3': 0.5286840442642281,
 's4': 6.099244267332785,
 's5': 0}

In [296]:
def estimate_occupancy(episodes, state, action, timestep_max, gamma):
    rho = 0
    total_times = np.zeros(timestep_max) # number of all (state, action) occurred in episode at timestep t
    occur_times = np.zeros(timestep_max) # times of (state, action) occurred in episode at timestep t
    for episode in episodes:
        for i in range(len(episode)):
            (s, a, r, s_next) = episode[i]
            total_times[i] += 1
            if state == s and action == a:
                occur_times[i] += 1
    for t in reversed(range(timestep_max)):
        if total_times[t] > 0:
            rho += (gamma ** t) * (occur_times[t] / total_times[t])
    
    return (1 - gamma) * rho

gamma = 0.5
timestep_max = 1000

episodes_pi1 = sample(MDP, Pi_1, timestep_max, 1000)
episodes_pi2 = sample(MDP, Pi_2, timestep_max, 1000)

rho_1 = estimate_occupancy(episodes_pi1, 's4', 'Go in Prob', timestep_max, gamma)
rho_2 = estimate_occupancy(episodes_pi2, 's4', 'Go in Prob', timestep_max, gamma)
print('Occupancy measure under Pi_1: %f' % rho_1)
print('Occupancy measure under Pi_2: %f' % rho_2)

Occupancy measure under Pi_1: 0.116357
Occupancy measure under Pi_2: 0.234478
